In [1]:
import numpy as np
import pandas as pd

import sys

from os.path import expanduser
sys.path.append(expanduser('~') + '/Lab/Utils/Python/')

from Conversions.translate import *
from Strings.is_a import *

# Initial setup

In [2]:
paper_pmid = 19416971
paper_name = 'khozoie_avery_2009' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/jbc.M109.005843-1.xls', sheet_name='Initial QN Screen', skiprows=3)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5664 x 13


In [7]:
original_data.head()

,ORF,Plate,Row,Col,Control A,Control B,Quinine A,Quinine B,MEAN Control,MEAN +Quinine,Growth Ratio (GR),Plate Median,Adjusted GR
0,EMPTY,1,A,1,0.117,0.116,0.110,0.110,0.1165,0.1100,SLOW,1.56586,SLOW
1,YHL047C,1,A,2,0.471,0.443,0.489,0.466,0.4570,0.4775,0.957068,1.56586,0.611209
2,YHL046C,1,A,3,0.736,0.509,0.406,0.454,0.6225,0.4300,1.44767,1.56586,0.924523
3,YHL045W,1,A,4,0.667,0.461,0.509,0.485,0.5640,0.4970,1.13481,1.56586,0.724719
4,YHL044W,1,A,5,0.117,0.119,0.111,0.110,0.1180,0.1105,SLOW,1.56586,SLOW


In [8]:
original_data['ORF'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['ORF'] = clean_orf(original_data['ORF'])

In [10]:
# Translate to ORFs 
original_data['ORF'] = translate_sc(original_data['ORF'], to='orf')

In [11]:
to_drop = original_data.loc[original_data['ORF']=='EMPTY',]

In [12]:
original_data.drop(index=to_drop.index, inplace=True)

In [13]:
# Make sure everything translated ok
t = looks_like_orf(original_data['ORF'])
print(original_data.loc[~t,])

                  ORF  Plate Row Col  Control A  ...  MEAN Control  MEAN +Quinine  Growth Ratio (GR)  Plate Median  Adjusted GR
index_input                                      ...                                                                           
5604         WTBY4743     71   D   1      0.082  ...        0.0815         0.0825               SLOW           NaN         SLOW
5605         WTBY4743     71   D   2      0.083  ...        0.0820         0.0840               SLOW           NaN         SLOW

[2 rows x 13 columns]


In [14]:
original_data.drop(index=original_data.loc[~t,:].index, inplace=True)

In [15]:
# Eliminate the slow growin strains for which no accurate growth ratio could be calculated
to_drop = original_data.loc[original_data['Adjusted GR']=='SLOW',]

In [16]:
original_data.drop(index=to_drop.index, inplace=True)

In [17]:
original_data.shape

(4394, 13)

In [25]:
# Reverse the growth ratio so that lower values correspond to decreased growth and viceversa (originally, GR is reported as untreated vs treated)
original_data['GR2'] = 1 / original_data['Growth Ratio (GR)']

In [26]:
# Normalize by plate median (as done oridinally)
def normalize_by_plate_median(plate_data):
    plate_median = plate_data['GR2'].median()
    plate_data['GR2_adjusted'] = plate_data['GR2'] / plate_median
    return plate_data

In [27]:
original_data2 = original_data.groupby('Plate').apply(normalize_by_plate_median)

In [28]:
original_data2.head()

,ORF,Plate,Row,Col,Control A,Control B,Quinine A,Quinine B,MEAN Control,MEAN +Quinine,Growth Ratio (GR),Plate Median,Adjusted GR,GR2,GR2_adjusted
index_input,,,,,,,,,,,,,,,
1,YHL047C,1,A,2,0.471,0.443,0.489,0.466,0.4570,0.4775,0.957068,1.56586,0.611209,1.04486,1.6361
2,YHL046C,1,A,3,0.736,0.509,0.406,0.454,0.6225,0.4300,1.44767,1.56586,0.924523,0.690763,1.08164
3,YHL045W,1,A,4,0.667,0.461,0.509,0.485,0.5640,0.4970,1.13481,1.56586,0.724719,0.881206,1.37984
5,YHL043W,1,A,6,0.659,0.576,0.495,0.497,0.6175,0.4960,1.24496,1.56586,0.795064,0.803239,1.25776
7,YHL041W,1,A,8,0.617,0.428,0.564,0.461,0.5225,0.5125,1.01951,1.56586,0.651088,0.980861,1.53589


In [40]:
data = original_data2[['ORF','GR2_adjusted']].copy()

In [41]:
data['GR2_adjusted'] = data['GR2_adjusted'].astype(float)

In [42]:
data.set_index('ORF', inplace=True)

# Prepare the final dataset

In [43]:
dataset_ids = [16533]

In [44]:
datasets = datasets.reindex(index=dataset_ids)

In [45]:
data.columns = datasets['name'].values

In [47]:
data = data.groupby(data.index).mean()

In [48]:
# Create row index
data.index.name='orf'

In [49]:
print('Final data dimensions: %d x %d' % (data.shape))

Final data dimensions: 4251 x 1


# Print out

In [50]:
data.to_csv(paper_name + '.txt', sep='\t')

# Save to DB

In [51]:
from IO.save_data_to_db2 import *

In [52]:
# Create column index
lst = [datasets.index.values, datasets['name'].values]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','dataset_name'])
data.columns = idx

In [53]:
save_data_to_db(data, paper_pmid)